In [0]:
pip install json5

Python interpreter will be restarted.
Collecting json5
 Downloading json5-0.9.5-py2.py3-none-any.whl (17 kB)
Installing collected packages: json5
Successfully installed json5-0.9.5
Python interpreter will be restarted.

In [0]:
#Use case
#Community Master Data: Users, Roles, UserRole
#Transaction Data: Transactions (transId, userId, some details)
#From userId, get the relevant username and perform anyother lookups

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg
from pyspark.sql.types import Row
import json5 as json

columns = ["RId","RName", "RActive"]
data = [(1, "Role1", 1),
    (2, "Role2", 1),
    (3, "Role3", 1)]
df_role = spark.createDataFrame(data=data, schema=columns)
df_role.createOrReplaceTempView("roles")
bc_role = spark.sparkContext.broadcast(df_role.toJSON().collect())

columns = ["UId","UName", "UActive"]
data = [(1, "User1", 1),
    (2, "User2", 1),
    (3, "User3", 0),
    (4, "User4", 1)]
df_user = spark.createDataFrame(data=data, schema=columns)
df_user.createOrReplaceTempView("users")
bc_user = spark.sparkContext.broadcast(df_user.toJSON().collect())

columns = ["UId", "RId"]
data = [(1, 1), (1, 2), (2, 3), (3, 1), (3,2), (3,3), (4, 1), (4,3)]
df_user_role = spark.createDataFrame(data=data, schema=columns)
df_user_role.createOrReplaceTempView("user_role")
bc_user_role = spark.sparkContext.broadcast(df_user_role.toJSON().collect())


def myMapFunction(rec, bc_user, bc_role, bc_user_role):
  print("myMapFunction::Type:%s, Data:%s" % (type(rec), rec))
  #myMapFunction::Type:<class 'pyspark.sql.types.Row'>, Data:Row(TId=4, UId=3, TDesc='TDesc4', TDate='2021-04-16T17:14:00')
  row_dict = rec.asDict()
  userId = row_dict.get("UId")
  print("myMapFunction::user:%s, bc_user:%s, bc_user.value:%s" % (userId, type(bc_user), type(bc_user.value)))
  enrich1_val = ""
  for v in bc_user.value:
    print("myMapFunction::type(v):%s, v:%s" % (type(v), v))
    #myMapFunction::type(v):<class 'str'>, v:{"UId":1,"UName":"U1","UActive":1}
    if type(v) == str:
      v = json.loads(v)
    if v.get("UId") == userId:
      enrich1_val = v.get("UName")
  #myMapFunction::user:3, bc_user:<class 'pyspark.broadcast.Broadcast'>, bc_user.value:<class 'list'>
  row_dict["ENRICH_1"] = enrich1_val #Adding a new column and setting some value.
  print("myMapFunction::row_dict:%s" % row_dict)
  #myMapFunction::row_dict:{'TId': 4, 'UId': 3, 'TDesc': 'TDesc4', 'TDate': '2021-04-16T17:14:00', 'ENRICH_1': 'constant_value'}
  return Row(**row_dict)

columns = ["TId", "UId", "TDesc", "TDate"]
data = [
  (1, 1, "TDesc1", "2021-04-16T17:11:00"), 
  (2, 1, "TDesc2", "2021-04-16T17:12:00"), 
  (3, 4, "TDesc3", "2021-04-16T17:13:00"), 
  (4, 3, "TDesc4", "2021-04-16T17:14:00"), 
  (5, 2, "TDesc5", "2021-04-16T17:15:00")]
df_trans = spark.createDataFrame(data=data, schema=columns)

df_new_rdd = df_trans.rdd.map(lambda r: myMapFunction(r, bc_user, bc_role, bc_user_role))
print("df_new_rdd:", df_new_rdd)
df_new_rdd.toDF(["TId", "UId", "TDesc", "TDate", "ENRICH_1"]).show()

df_new_rdd: PythonRDD[1078] at RDD at PythonRDD.scala:58
+---+---+------+-------------------+--------+
TId|UId| TDesc| TDate|ENRICH_1|
+---+---+------+-------------------+--------+
 1| 1|TDesc1|2021-04-16T17:11:00| User1|
 2| 1|TDesc2|2021-04-16T17:12:00| User1|
 3| 4|TDesc3|2021-04-16T17:13:00| User4|
 4| 3|TDesc4|2021-04-16T17:14:00| User3|
 5| 2|TDesc5|2021-04-16T17:15:00| User2|
+---+---+------+-------------------+--------+